In [ ]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kmf11\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%cd gdrive/MyDrive/Datasets

/content/gdrive/MyDrive/Datasets


In [ ]:
import os 
for file in os.listdir():
  if file.endswith(".csv"):
    import pandas as pd 
    data = pd.read_csv(file, encoding='latin', header=None)

In [ ]:
!python -m spacy download en_core_web_sm

2022-10-26 16:44:37.067012: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 21.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
%pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import pandas as pd 
import numpy as np 
from torch import nn
import  torch.nn.functional as f
import spacy
import nltk
from nltk.corpus import wordnet as wn
import torchtext as tt
from nltk.stem import SnowballStemmer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.shape

(1600000, 6)

In [ ]:
data.columns

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [ ]:
data[3].value_counts()

NO_QUERY    1600000
Name: 3, dtype: int64

In [ ]:
data.drop(3, axis=1, inplace=True)

In [ ]:
data.rename({0:'target'}, axis=1, inplace=True)

In [ ]:
data.drop(1, axis=1, inplace=True)

In [ ]:
X = data[5]
Y = data['target']

X.shape, Y.shape

((1600000,), (1600000,))

In [ ]:
Y.unique()

array([0, 4])

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
class SentimentAnalysis(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(SentimentAnalysis, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim , rnn_units)
        self.clf = nn.Linear(rnn_units, 1)


    
    def forward(self, x):
        r = self.embedding(x)
        r = self.lstm1(r)
        r = self.clf(r)
        
        return 



        
        

In [ ]:
from sklearn.model_selection import train_test_split 
X = X.apply(preprocessor)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
def preprocessor(text):
    import  re
    from nltk import LancasterStemmer
    ls = LancasterStemmer()
    pattern = r"http"
    pattern2 = r"[.,//:;?!-#]"
    x1 = " ".join([i if pattern not in i and "@" not in i and len(i) > 1 else "" for i in text.split()])
    x2 = nlp(x1)
    x3 = " ".join([ls.stem(word.lemma_) for word in x2])
    final = re.sub(pattern2, " ", x3)
    
    return final



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf = Pipeline(
    [("vectorizer_bow", CountVectorizer()),
     ("multinomial_nb", MultinomialNB())
    ]
)

clf.fit(X_train.values, Y_train)

y_hat = clf.predict(X_test)

print(classification_report(Y_test, y_hat))